In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

irisDF = pd.read_csv("../datas/iris.csv")
irisNP = np.loadtxt("../datas/iris.csv",dtype=float, delimiter=',', usecols=[0, 1, 2, 3])

ValueError: could not convert string 'sepal.length' to float64 at row 0, column 1.

In [ ]:
class DLDataset(TensorDataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        self.feature=torch.FloatTensor(x_data)
        self.target=torch.LongTensor(y_data)
    def __len__(self):
        return self.target.shape[0]
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [ ]:
feature = irisDF[irisDF.columns[:-1]]
target = irisDF[irisDF.columns[-1]]

from sklearn.preprocessing import LabelEncoder
target = LabelEncoder().fit_transform(target)
target=target.reshape(-1,1)
my_dataset=DLDataset(feature, target)

In [ ]:
from torch.utils.data import random_split

seed=torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset, [0.7,0.1,0.2], generator=seed)

In [ ]:
batchsize=5
trainDF=DataLoader(trainDS, batch_size=batchsize, shuffle=True)
validDF=DataLoader(validDS, batch_size=batchsize, shuffle=True)
testDF=DataLoader(testDS, batch_size=batchsize, shuffle=True)

In [ ]:
class model(nn.Module) :
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(4, 3)
    def forward(self, x):
        x = self.fc1(x)
        return x
    
model=model()

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS = 50
LOSS_FN=nn.CrossEntropyLoss().to(DEVICE)

In [ ]:
optimizer=optim.Adam(model.parameters(), lr=0.001)

def training():
    for epoch in range(EPOCHS):
        model.train()
        for cnt, (feature, target) in enumerate(trainDF):
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            pre_target = model(feature.float())
            loss=LOSS_FN(pre_target, target.squeeze())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
